<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/climate/climate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install netCDF4 basemap --quiet

In [ ]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import PIL
from datetime import datetime, timedelta, date
from mpl_toolkits.basemap import Basemap
import os

In [10]:
ds = Dataset("drive/MyDrive/climate/sliced_datasets/tg_ens_mean_0.1deg_reg_1965-1979_v25.0e.nc_sliced")

lats = ds.variables['latitude'][:]
lons = ds.variables['longitude'][:]
dates = ds.variables['time'][:]
rain = ds.variables['tg'][:]

lon, lat = np.meshgrid(lons, lats)

In [35]:
#longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
start_date = date(1965, 1, 1)

days = 0

for i in range(10):
  end_date = start_date + timedelta(days=days)
  y, m, d = end_date.strftime("%Y"), end_date.strftime("%m"), end_date.strftime("%d")
  end_date_str = y + "-" + m + "-" + d

  map = Basemap(llcrnrlon=16.06, llcrnrlat=45.65, urcrnrlon=23.02, urcrnrlat=48.63, projection='merc', resolution='i')

  x,y = map(lon, lat)
  map.drawcountries()

  plt.title("Napi átlag hőmérséklet: {0}".format(end_date_str))

  color_scheme = map.pcolormesh(x, y, np.squeeze(rain[i, :, :]), cmap='jet', vmin=-30, vmax=40)
  color_bar = map.colorbar(color_scheme, location='right', pad='5%')
  color_bar.set_label('Napi átlag hőmérséklet (°C)', labelpad=5)
  plt.savefig("drive/MyDrive/climate/avg_temp/{0}.jpg".format(days), dpi=300)
  plt.clf()
  days += 1


<Figure size 432x288 with 0 Axes>

In [ ]:
frames = []

for i in range(1000, 10000):
  frame = PIL.Image.open("drive/MyDrive/climate/avg_temp/{0}.jpg".format(str(i+1)))
  frames.append(frame)

frames[0].save('drive/MyDrive/climate/gifs/output-temp2.gif', 
               format='GIF',
               append_images = frames[1: ], 
               save_all = True,
               duration=20, loop=0)

In [ ]:
dir = "drive/MyDrive/climate"
for f in os.listdir(dir):
  fname = os.path.join(dir, f)
  if os.path.isfile(fname):
    ds = xr.open_dataarray(fname)
    ds = ds.sel(time=ds['time'], longitude=slice(15.96, 23.12), latitude=slice(45.55, 48.73))
    ds.to_netcdf("drive/MyDrive/climate/sliced_datasets/{0}_sliced".format(f))
    #print(fname)